# 1. 라이브러리 임포트

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
import warnings
import matplotlib.pyplot as plt
import pandas as pd 
import joblib, os

import warnings
import platform
import matplotlib.pyplot as plt

from matplotlib import font_manager, rc 
warnings.filterwarnings('ignore')

path = "C:/Windows/Fonts/malgun.ttf"

if platform.system() == "Darwin":
    rc("font", family="Arial Unicode MS")
elif platform.system() == "Windows":
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc("font", family=font_name)
else:
    print("Unknown system. sorry")

C:\Users\User\anaconda3\envs\tf\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# 2. 데이터 로드

사용할 Feature 선택

In [55]:
m = ['url','content','context_img','content_cnt','content_line', 'img_cnt','... 빈도 수','내돈내산 키워드','쿠팡키워드','ㅠ 빈도 수',
     '광고키워드','단점 빈도 수','ㅠ 빈도 수','광고키워드','단점 빈도 수','이미지광고키워드3','비교 빈도 수',
     '솔직 빈도 수','coupan.ng 키워드','quote_cnt','ㅋㅋㅋㅋ 빈도 수','ㅋ 빈도 수','이미지글없음','이미지광고키워드1',
     '이미지쿠팡키워드1','ㅋㅋ 빈도 수','광고 분류1', '광고 분류2'
]

In [56]:
df = pd.read_csv("Total_Data_4(1).csv")
df = df[m]
df.head(2)

,url,content,context_img,content_cnt,content_line,img_cnt,... 빈도 수,내돈내산 키워드,쿠팡키워드,ㅠ 빈도 수,...,coupan.ng 키워드,quote_cnt,ㅋㅋㅋㅋ 빈도 수,ㅋ 빈도 수,이미지글없음,이미지광고키워드1,이미지쿠팡키워드1,ㅋㅋ 빈도 수,광고 분류1,광고 분류2
0,https://blog.naver.com/PostView.naver?blogId=s...,오랜만에 다시 돌아온 또니로그😆 코닥 미니샷 3 선물받고 신난 마음에 블로그 들어왔...,the,617,13,11,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,https://blog.naver.com/PostView.naver?blogId=h...,사방팔방 아기들에게 생일선물로 받은 즉석카메라 코닥 미니샷 콤보3 레트로!! 간단하...,NaN,1305,19,19,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0


# 3. Feature 생성

## 3.1 OCR에서 뽑힌 이미지 데이터에 특징 만들기
### 이전에 비해서 적중률을 높게 한다면 어떻게 할까 라는 고민 끝에 최종적인 feature 생성하기로 함
> 제품을 제공받아 작성된 글입니다.

라고 하면 제품 1, 제공 1 작성 1 받아 1 -> 총 4점을 측정시켜, 점수가 고점일 수록 광고일 확률이 높다 라고 가설을 세워서 진행하고자 한다.

In [58]:
df.loc[df['context_img'].isnull(),"context_img"] = ''

* 광고 적중률(AD_score)

In [57]:
kk_list = ['부터|에서','반드시','제공', '대여' ,'무상','받을|밥아|받아|반아|바아|받이|반이','선정|선점' ,'지원' ,'업체' ,'원고료|운고료','협찬|협진|협잔','부터', '소정료' ,'취재비' ,'제품','서비스' ,'체험단|제험단|체험' ,'제작비|제작|삭성|제삭|체삭|세작|새작|재작']

In [59]:
col_list_ad = []
for i in range(len(kk_list)):
#     print(kk_list[i])
    save_col = f"이미지광고_{i}"
    df[save_col] = 0
    df.loc[df["context_img"].str.contains(kk_list[i]),save_col]=1
    col_list_ad.append(save_col)

In [60]:
df["AD_score"] = df[col_list_ad].sum(axis=1)

In [61]:
df[df["AD_score"]>1]["AD_score"].describe()

count    130.000000
mean       4.123077
std        1.452159
min        2.000000
25%        3.000000
50%        4.000000
75%        5.000000
max        7.000000
Name: AD_score, dtype: float64

In [62]:
df[["광고 분류2","AD_score"]].groupby(["광고 분류2"]).describe()

AD_score                                             
          count      mean       std  min  25%  50%  75%  max
광고 분류2                                                      
0        1932.0  0.038820  0.243069  0.0  0.0  0.0  0.0  5.0
1         177.0  0.169492  0.482155  0.0  0.0  0.0  0.0  2.0
2         361.0  1.443213  2.145464  0.0  0.0  0.0  3.0  7.0

* 쿠팡 파트너스 적중률(CP_score)

In [67]:
kk_list = ['파트너스|피트너스|피트|너스','쿠팡|쿠광|구팡|구파|쿠파','일정액|수수료|이정액|수료','수익|수이|스이|스익','일원|일워|인워']

In [68]:
col_list_cupan = []
for i in range(len(kk_list)):
#     print(kk_list[i])
    save_col = f"이미지쿠팡_{i}"
    df[save_col] = 0
    df.loc[df["context_img"].str.contains(kk_list[i]),save_col]=1
    col_list_cupan.append(save_col)

In [69]:
df["CP_score"] = df[col_list_cupan].sum(axis=1)

In [70]:
df[["광고 분류2","CP_score"]].groupby(["광고 분류2"]).describe()

CP_score                                             
          count      mean       std  min  25%  50%  75%  max
광고 분류2                                                      
0        1932.0  0.002070  0.071933  0.0  0.0  0.0  0.0  3.0
1         177.0  0.265537  0.777963  0.0  0.0  0.0  0.0  4.0
2         361.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0

* 순수 리뷰 적중률 (PU_score)

In [71]:
kk_list = ['내돈내산|내돈|내산','사비','직접|지접','구매|그매','일정액|일정','솔직','않은|않']

In [72]:
col_list_Pu = []
for i in range(len(kk_list)):
#     print(kk_list[i])
    save_col = f"이미지순수_{i}"
    df[save_col] = 0
    df.loc[df["context_img"].str.contains(kk_list[i]),save_col]=1
    col_list_Pu.append(save_col)

In [73]:
df["PU_score"] = df[col_list_Pu].sum(axis=1)

In [74]:
df[["광고 분류2","PU_score"]].groupby(["광고 분류2"]).describe()

PU_score                                             
          count      mean       std  min  25%  50%  75%  max
광고 분류2                                                      
0        1932.0  0.024327  0.187458  0.0  0.0  0.0  0.0  3.0
1         177.0  0.141243  0.349260  0.0  0.0  0.0  0.0  1.0
2         361.0  0.038781  0.207211  0.0  0.0  0.0  0.0  2.0

In [75]:
df.drop(col_list_ad,axis=1,inplace=True)
df.drop(col_list_cupan,axis=1,inplace=True)
df.drop(col_list_Pu,axis=1,inplace=True)

In [78]:
df.columns

Index(['url', 'content', 'context_img', 'content_cnt', 'content_line',
       'img_cnt', '... 빈도 수', '내돈내산 키워드', '쿠팡키워드', 'ㅠ 빈도 수', '광고키워드',
       '단점 빈도 수', 'ㅠ 빈도 수', '광고키워드', '단점 빈도 수', '이미지광고키워드3', '비교 빈도 수',
       '솔직 빈도 수', 'coupan.ng 키워드', 'quote_cnt', 'ㅋㅋㅋㅋ 빈도 수', 'ㅋ 빈도 수',
       '이미지글없음', '이미지광고키워드1', '이미지쿠팡키워드1', 'ㅋㅋ 빈도 수', '광고 분류1', '광고 분류2',
       'AD_score', 'CP_score', 'PU_score'],
      dtype='object')

In [77]:
df.to_csv("Total_Data_5(1).csv",index=False)